In [83]:
pip install -r setup.txt

In [9]:
#from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU

import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
import librosa as lr
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from librosa.display import specshow
from os import path

In [10]:
train_data_path='518h0245_518h0439/data/train/'
test_data_path='518h0245_518h0439/data/test/'
wav_path = '518h0245_518h0439/data/wav/train'
wav_path_test = '518h0245_518h0439/data/wav/test'

In [81]:
def create_spectrogram(filename,name, file_path):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = file_path + name + '.png'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [82]:

Data_dir=glob(wav_path+"/*.wav")

for file in Data_dir: 
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name, train_data_path)
    
gc.collect()

AttributeError: ignored

In [13]:
Test_dir=glob(wav_path_test+"/*.wav")

for file in Test_dir:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name,test_data_path)

gc.collect()


18502

In [84]:
def append_ext(fn):
    return fn.replace(".wav",".png")

# Load du lieu train va test tu file csv
traindf=pd.read_csv('518h0245_518h0439/data/train.csv',dtype=str)
testdf=pd.read_csv('518h0245_518h0439/data/test.csv',dtype=str)
traindf["slice_file_name"]=traindf["slice_file_name"].apply(append_ext)
testdf["slice_file_name"]=testdf["slice_file_name"].apply(append_ext)

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
print(traindf)



   slice_file_name    fsID start   end salience fold classID class
0         Bac1.png  100263  58.5  62.5        1    5       2   bac
1         Bac2.png  100263  60.5  64.5        1    5       2   bac
2         Bac3.png  100263    63    67        1    5       2   bac
3         Bac4.png  100263  68.5  72.5        1    5       2   bac
4         Bac5.png  100263  71.5  75.5        1    5       2   bac
5         Hue1.png  100852  80.5  84.5        1    5       1   hue
6         Hue2.png  100852   1.5   5.5        1    5       1   hue
7         Hue3.png  100852    18    22        1    5       1   hue
8         Hue4.png  100852     0     4        1    5       1   hue
9         Hue5.png  100852   0.5   4.5        1    5       1   hue
10        Nam6.png  100851     5     9        1    5       0   nam
11        Nam7.png  100851   5.5   9.5        1    5       0   nam
12        Nam1.png  100852   6.5  10.5        2    6       1   nam
13        Nam2.png  100853   7.5  11.5        3    7       2  

In [15]:
train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_data_path,
    x_col="slice_file_name",
    y_col="class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))
print(train_generator)

Found 16 validated image filenames belonging to 3 classes.


In [16]:
valid_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_data_path,
    x_col="slice_file_name",
    y_col="class",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

Found 5 validated image filenames belonging to 3 classes.


In [72]:
import tensorflow as tf
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error

In [73]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['Accuracy','mse'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_90 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 activation_105 (Activation)  (None, 64, 64, 32)       0         
                                                                 
 conv2d_91 (Conv2D)          (None, 62, 62, 64)        18496     
                                                                 
 activation_106 (Activation)  (None, 62, 62, 64)       0         
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 31, 31, 64)       0         
 g2D)                                                            
                                                                 
 dropout_60 (Dropout)        (None, 31, 31, 64)        0         
                                                     

In [77]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN)
print(train_generator)

0


In [78]:
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
print(valid_generator)

In [79]:
import pickle
history=model.fit(train_generator, epochs=10, validation_data=valid_generator) 
model.save("model.h5");
with open('model_indices.pickle', 'wb') as handle:
    pickle.dump(train_generator.class_indices, handle, protocol=pickle.HIGHEST_PROTOCOL)

Epoch 1/10
1/1 [==============================] - 1s 742ms/step - loss: 0.5529 - Accuracy: 0.8750 - mse: 0.0923 - val_loss: 1.6089 - val_Accuracy: 0.0000e+00 - val_mse: 0.3325
Epoch 2/10
1/1 [==============================] - 1s 646ms/step - loss: 0.4142 - Accuracy: 0.8750 - mse: 0.0706 - val_loss: 1.5637 - val_Accuracy: 0.0000e+00 - val_mse: 0.3522
Epoch 3/10
1/1 [==============================] - 1s 645ms/step - loss: 0.4034 - Accuracy: 0.8750 - mse: 0.0746 - val_loss: 2.2415 - val_Accuracy: 0.0000e+00 - val_mse: 0.5057
Epoch 4/10
1/1 [==============================] - 1s 654ms/step - loss: 0.8316 - Accuracy: 0.8125 - mse: 0.1298 - val_loss: 1.9106 - val_Accuracy: 0.0000e+00 - val_mse: 0.4153
Epoch 5/10
1/1 [==============================] - 1s 641ms/step - loss: 0.6164 - Accuracy: 0.8750 - mse: 0.0805 - val_loss: 1.2964 - val_Accuracy: 0.0000e+00 - val_mse: 0.2747
Epoch 6/10
1/1 [==============================] - 1s 633ms/step - loss: 0.4578 - Accuracy: 0.8125 - mse: 0.0817 - val_lo

In [85]:
from keras.models import load_model
from keras_preprocessing.image import ImageDataGenerator
def append_ext(fn):
    return fn.replace(".wav",".png")

# Load du lieu test
testdf=pd.read_csv('518h0245_518h0439/data/test.csv',dtype=str)
testdf["slice_file_name"]=testdf["slice_file_name"].apply(append_ext)
test_data_path='518h0245_518h0439/data/test/'

# Khoi tao du lieu test
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory=test_data_path,
    x_col="slice_file_name",
    y_col=None,
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(64,64))

# TInh so buoc test
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

# Tien hanh predict
test_generator.reset()

# Load model da train
model = load_model('model.h5')
pred = model.predict(test_generator,verbose=1)


# Lay class predict probality lon nhat
predicted_class_indices=np.argmax(pred,axis=1)
# Load class name tu file
with open('model_indices.pickle', 'rb') as handle:
    labels = pickle.load(handle)

# HIen thi ket qua predict ra man hinh
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
print("Prediction values= ",predictions[0:15])
print("Real values=",testdf.head(15)["class"])

Found 9 validated image filenames.
1/1 [==============================] - 0s 223ms/step
Prediction values=  ['nam', 'hue', 'hue', 'hue', 'hue', 'hue', 'hue', 'nam', 'nam']
Real values= 0    nam
1    bac
2    bac
3    bac
4    hue
5    hue
6    hue
7    nam
8    nam
Name: class, dtype: object
